In [1]:
from datasets import load_dataset
import json

import random
from transformers import set_seed
import re

# Load the IMDB dataset
# imdb_dataset = load_dataset('imdb')

In [2]:
# Print the number of examples in the dataset
# print(f"Number of training examples: {len(imdb_dataset['train'])}")
# print(f"Number of test examples: {len(imdb_dataset['test'])}")

In [3]:
# train_dataset = imdb_dataset['train']
# test_dataset = imdb_dataset['test']

In [4]:
# convert to json and export into raw folder
# train_dataset.to_json("../data/raw/train_imdb.json")
# test_dataset.to_json("../data/raw/test_imdb.json")

In [5]:
# load json files
train_dataset = []
with open("train_imdb.json") as f:
    for line in f:
        train_dataset.append(json.loads(line))

test_dataset = []
with open("test_imdb.json") as f:
    for line in f:
        test_dataset.append(json.loads(line))

In [6]:
train_dataset[0:2]

[{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far b

In [7]:
test_dataset[0:2]

[{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as

In [8]:
for item in train_dataset:
    item['text'] = re.sub(r"<br />", " ", item['text'])  # replaces all "<br />" with a white space
    item['text'] = " ".join(item['text'].split())  # makes sure there is only 1 white space between words

In [9]:
for item in test_dataset:
    item['text'] = re.sub(r"<br />", " ", item['text'])  # replaces all "<br />" with a white space
    item['text'] = " ".join(item['text'].split())  # makes sure there is only 1 white space between words

In [10]:
# randomly sample 100 from train_dataset and randomly sample 1000 from test_dataset
set_seed(123)

random.shuffle(train_dataset)
train_dataset = train_dataset[:100]

random.shuffle(test_dataset)
test_dataset = test_dataset[:1000]

In [11]:
# Create a shuffled list of sentence2 values for train_dataset
shuffled_sentence2 = [item['label'] for item in train_dataset]
random.shuffle(shuffled_sentence2)

# Assign shuffled values to sentence2 key in each dictionary in train_dataset
for i, item in enumerate(train_dataset):
    item['sentence2'] = shuffled_sentence2[i]

In [12]:
# Create a shuffled list of sentence2 values for test_dataset
shuffled_sentence2 = [item['label'] for item in test_dataset]
random.shuffle(shuffled_sentence2)

# Assign shuffled values to sentence2 key in each dictionary in test_dataset
for i, item in enumerate(test_dataset):
    item['sentence2'] = shuffled_sentence2[i]

In [15]:
# rename 'text' column to 'sentence1'
for item in train_dataset:
    item['sentence1'] = item.pop('text')

for item in test_dataset:
    item['sentence1'] = item.pop('text')

In [17]:
# modify the label column to "entailment" or "contradiction"
for item in train_dataset:
    if item['label'] == item['sentence2']:
        item['label'] = "entailment"
    else:
        item['label'] = "contradiction"

for item in test_dataset:
    if item['label'] == item['sentence2']:
        item['label'] = "entailment"
    else:
        item['label'] = "contradiction"

In [19]:
# modify the sentence2 column to prompts instead of classes
for item in train_dataset:
    if item['sentence2'] == 0:
        item['sentence2'] = "This movie review is negative."
    else:
        item['sentence2'] = "This movie review is positive."

for item in test_dataset:
    if item['sentence2'] == 0:
        item['sentence2'] = "This movie review is negative."
    else:
        item['sentence2'] = "This movie review is positive."

In [22]:
train_dataset

[{'label': 'contradiction',
  'sentence2': 'This movie review is negative.',
  'sentence1': "I've heard people who say this movie is dull dull dull. I don't think they were watching the right movie. This isn't the prototypical action movie, thank God. This is a psychological drama about the rookie and his mentor that just happens to be about killing people. In this way it works extremely well, with terrific performances from Berenger and Zane (who doesn't sleepwalk through the movie like he has in other roles - he actually looks like he's acting). I was disappointed with the action towards the end - a lot of it didn't make much sense and was unsatisfying given the buildup from the rest of the movie. But watch Zane's face as he panics, alone, while Berenger does the dirty work."},
 {'label': 'entailment',
  'sentence2': 'This movie review is positive.',
  'sentence1': "Three flash-backs introduce the main characters (Abu, Jaffar, and the Princess) who will interact with Ahmad; three are

In [141]:
eval_dataset = test_dataset[0:500]
test_dataset = test_dataset[500:1000]